In [ ]:
import os
import geopandas.datasets
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as geopd
import sklearn.cluster as clustering
import numpy as np
from matplotlib.ticker import FormatStrFormatter, PercentFormatter
os.chdir("/Volumes/ExtraHDD2/DS_Assignments_Data/ADM_2")

In [ ]:
posts = pd.read_csv("instagram_posts.csv", delimiter="\t", parse_dates=[5], infer_datetime_format = True)

In [ ]:
# RQ2 [H1]

# Most common time in which users publish their posts [H2]

# This is pretty straightforward to do
# It should be stressed that we have no information from the dataset for what concerns the timezones, which is a quite crucial bit of info
# The timestamps probably refer to a single timezone, so it is difficult to talk about the "most common time", since to check
# something like this we should have the timestamps relatively to the timezone of the account which posted.
# Considering the limitation, let's answer the question
print((posts.cts.dt.hour.value_counts(True)*100).astype("str").add("%"))
plt.figure(figsize=(9, 6))
posts.cts.dt.hour.value_counts(True).sort_index().plot.bar()
plt.xlabel("Hour")
plt.ylabel("Percentage")
plt.title("Barplot for posts frequency conditioned on time")
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('% .0f'))
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))
plt.show()
# The time range which seems more active is the one from early-mid afternoon to 9/10 PM.

In [ ]:
# Define a function that receives a time interval and returns a plot with the number of posts for each time interval